In [64]:
desired_cube_side = 512
chunk_size = 64
# raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF'
# Has issues on SciServer Compute
use_dask = False
dest_folder_name = "sabl2048a"
# timestep_nr = 1
# timestep_range = range(2,6)

In [65]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [73]:
import xarray as xr
import os
from utils import write_tools
import dask
import numpy as np

In [85]:
z_order = np.array(write_tools.morton_order_cube(4))

In [86]:
z_order[0,:,:]

array([[ 0,  4, 32, 36],
       [ 2,  6, 34, 38],
       [16, 20, 48, 52],
       [18, 22, 50, 54]])

In [75]:
data_xr = xr.open_dataset("/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF/jhd.001.nc")

In [76]:
data_xr.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:  (nnx: 2048, nny: 2048, nnz: 2048)
Dimensions without coordinates: nnx, nny, nnz
Data variables:
    u        (nnz, nny, nnx) float32 ...
    v        (nnz, nny, nnx) float32 ...
    w        (nnz, nny, nnx) float32 ...
    t        (nnz, nny, nnx) float32 ...
    p        (nnz, nny, nnx) float32 ...
    e        (nnz, nny, nnx) float32 ...
Attributes:
    Simulation conducted by:  Peter Sullivan, pps@ucar.edu
    Dataset built by:         Edward Patton, patton@ucar.edu
    Affiliation:              NCAR/MMM
    Code:                     NCAR-LES
    Case Description:         GABLS 1, Cooling Rate: 0.25 K/hr
    Created:                  2023-05-02 10:34:38 -0600 UTC>

In [ ]:
np.array(data_xr['p'][:100][:100][:100])

In [62]:
dims = [dim for dim in data_xr.dims]
dims

['nnx', 'nny', 'nnz']

In [5]:
# Split 2048^3 into smaller 512^3 arrays
dims = [dim for dim in data_xr.dims]
smaller_groups = write_tools.split_zarr_group(data_xr, desired_cube_side, dims)

In [26]:
ds = data_xr
smaller_size = desired_cube_side
i=0
j=0
k=0


chunk = ds.isel( # TODO make this into a function: slice_group()
    {dims[0]: slice(0, 512),
     dims[1]: slice(0, 512),
     dims[2]: slice(512, 1024)}
)

In [25]:
dims[2]

'nnz'

In [27]:
import numpy as np

np.testing.assert_array_equal(
    chunk['e'],#.to_array,
    smaller_groups[0][0][1]['e']
)

In [23]:
len(smaller_groups[0][0])

4

In [30]:
smaller_groups[0][0][0]['e'].shape

(512, 512, 512)

In [34]:
a = tuple(element * desired_cube_side for element in (1,1,1))
a

(512, 512, 512)

In [36]:
a + (512,512,512)

(512, 512, 512, 512, 512, 512)

In [50]:
import morton, math
array_cube_side = 2048


bits = int(math.log(array_cube_side, 2))
mortoncurve = morton.Morton(dimensions = 3, bits = bits)

mortoncurve.pack(0,0,1)
mortoncurve.unpack(134217727)

[511, 511, 511]

In [52]:
mortoncurve.unpack(134217728)

[512, 0, 0]

In [60]:
mortoncurve.pack(1, 0, 0) * 512 ** 3

134217728

In [59]:
mike_dict = {}
for i in range(len(smaller_groups)):
    for j in range(len(smaller_groups[i])):
        for k in range(len(smaller_groups[i][j])):
            min_coord = tuple(element * desired_cube_side for element in (i,j,k))
            max_coord = tuple((element * desired_cube_side)-1 for element in (i+1,j+1,k+1))
            
            point_on_cube = (i * 4 * 4) + (j * 4) + k # 0 to 63
            print(point_on_cube + 1, (i,j,k))
            
#             mike_dict[dest_folder_name + str(point_on_cube + 1).zfill(2)] = (write_tools.morton_pack(2048, min_coord[0], min_coord[1], min_coord[2]), write_tools.morton_pack(2048, max_coord[0], max_coord[1], max_coord[2]))
#             print(write_tools.morton_pack(2048, min_coord[0], min_coord[1], min_coord[2]))
# print(mike_dict)

1 (0, 0, 0)
2 (0, 0, 1)
3 (0, 0, 2)
4 (0, 0, 3)
5 (0, 1, 0)
6 (0, 1, 1)
7 (0, 1, 2)
8 (0, 1, 3)
9 (0, 2, 0)
10 (0, 2, 1)
11 (0, 2, 2)
12 (0, 2, 3)
13 (0, 3, 0)
14 (0, 3, 1)
15 (0, 3, 2)
16 (0, 3, 3)
17 (1, 0, 0)
18 (1, 0, 1)
19 (1, 0, 2)
20 (1, 0, 3)
21 (1, 1, 0)
22 (1, 1, 1)
23 (1, 1, 2)
24 (1, 1, 3)
25 (1, 2, 0)
26 (1, 2, 1)
27 (1, 2, 2)
28 (1, 2, 3)
29 (1, 3, 0)
30 (1, 3, 1)
31 (1, 3, 2)
32 (1, 3, 3)
33 (2, 0, 0)
34 (2, 0, 1)
35 (2, 0, 2)
36 (2, 0, 3)
37 (2, 1, 0)
38 (2, 1, 1)
39 (2, 1, 2)
40 (2, 1, 3)
41 (2, 2, 0)
42 (2, 2, 1)
43 (2, 2, 2)
44 (2, 2, 3)
45 (2, 3, 0)
46 (2, 3, 1)
47 (2, 3, 2)
48 (2, 3, 3)
49 (3, 0, 0)
50 (3, 0, 1)
51 (3, 0, 2)
52 (3, 0, 3)
53 (3, 1, 0)
54 (3, 1, 1)
55 (3, 1, 2)
56 (3, 1, 3)
57 (3, 2, 0)
58 (3, 2, 1)
59 (3, 2, 2)
60 (3, 2, 3)
61 (3, 3, 0)
62 (3, 3, 1)
63 (3, 3, 2)
64 (3, 3, 3)
